In [1]:
import math
import nltk  ## Natural Language Tool Kit
from sklearn.externals import joblib
from tkinter import *
import tkinter.messagebox
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
Dir = "D:/python_enviroments/PythonCodes/数据挖掘/期末作业/"
pklDir = "D:/python_enviroments/PythonCodes/数据挖掘/期末作业/pkl/"
sparce_matrix=joblib.load(pklDir+'sparce_matrix.pkl')
text_list = joblib.load(pklDir+'text_list.pkl')
cou_vec=CountVectorizer(max_features=1000)
def text_standardize(text_data):
    text_list=[]
    list_stopwords=list(set(stopwords.words('english')))
    text=re.sub("[^a-zA-Z]"," ",text_data)
    text=text.lower()
    text=nltk.word_tokenize(text)
    lemma=nltk.WordNetLemmatizer()
    text=[lemma.lemmatize(word) for word in text]
    text=[w for w in text if not w in list_stopwords]
    text=" ".join(text)
    text_list.append(text)
    return text_list

def judge_emotion(text,dictionary):
    test_result = []
    # 打开文件，并处理符号和停顿字符
    review = text_standardize(text)[-1]
    review = nltk.word_tokenize(review)
    # 将ham_score和spam_score置零
    score = [0,0,0]
    # 遍历文件中所有的单词，如果出现在字典中，那么计算他们的得分情况，对e求对数
    # 此处由于被分到垃圾类和非垃圾类概率一样，词频分布一样，故只需累乘即可
    for i in range(3):
        for word in review:
            if word in dictionary[i].keys():
                score[i]+=math.log(dictionary[i][word])
            else:
                score[i]+=math.log(0.05)
    positive_score = score[0]
    negative_score = score[1]
    neutral_score = score[2]         
    max_score = max(positive_score,negative_score,neutral_score)
    if positive_score == max_score:
        classify_label = 0
    elif negative_score == max_score:
        classify_label = 1
    else:
        classify_label = 2
    return classify_label

def drop_symbols(first_sample):
    temp = re.sub("[^a-zA-Z]"," ",first_sample)
    temp = temp.lower()
    temp=nltk.word_tokenize(temp)
    lemma_temp=nltk.WordNetLemmatizer()
    temp=[lemma_temp.lemmatize(word) for word in temp]
    temp=" ".join(temp)
    text_list.append(temp)
    sparce_matrix_temp=cou_vec.fit_transform(text_list).toarray()
    return [sparce_matrix_temp[-1]]
predict_method = joblib.load(pklDir+'lr.pkl')
method_number = 0

def get_method(num):
    global predict_method
    global method_number
    if num == 0:
        predict_method = joblib.load(pklDir+'rf.pkl')
        method_number = 0
    elif num == 1:
        predict_method = joblib.load(pklDir+'lr.pkl')
        method_number = 1
    else:
        predict_method = joblib.load(pklDir+'laplace_probability.pkl')
        method_number = 2

# 初始化Tk()
myWindow = Tk()
# 设置标题
myWindow.title("评论情感分析")
# 设置窗口大小
width = 300
height = 380
screenwidth = myWindow.winfo_screenwidth()
screenheight = myWindow.winfo_screenheight()
alignstr = '%dx%d+%d+%d' % (width, height, (screenwidth-width)/2, (screenheight-height)/2)
myWindow.geometry(alignstr)
# 设置窗口是否可变长、宽，True:可变，False:不可变
myWindow.resizable(False,False)
v=IntVar()
method=[('随机森林',0),('逻辑回归',1),('朴素贝叶斯',2)]
#定义单选按钮的响应函数
Label(myWindow,bg = 'blue',text='选择一门你所使用的预测算法：')
#for循环创建单选框
predict_logo = PhotoImage(file=Dir+"预测.gif")
bg = PhotoImage(file=Dir+"bg.gif")
imgLabel = Label(myWindow,compound=CENTER,image=bg).pack()
count = 0
for method,command in method:
    Radiobutton(myWindow, text=method,value=command,command=get_method(command),font=('Arial 12 bold'),variable=v).place(x=10,y=10+40*count)
    count+=1
label1=Label(myWindow,text="输入你的评论：",font=('Arial 12 bold'))
entry1=Entry(myWindow,width=40)
label1.place(x=10,y=150)
entry1.place(x=10,y=200)

def get_result():
    com=str(entry1.get())
    if method_number != 2:
        result = predict_method.predict(drop_symbols(com))
        if result[-1]==0:
            tc=tkinter.messagebox.showinfo(title='result', message = '积极')
        elif result[-1]==1:
            tc=tkinter.messagebox.showinfo(title='result', message = '消极')
        else:
            tc=tkinter.messagebox.showinfo(title='result', message = '中立')
    else:
        result = judge_emotion(com,predict_method)
        if result==0:
            tc=tkinter.messagebox.showinfo(title='result', message = '积极')
        elif result==1:
            tc=tkinter.messagebox.showinfo(title='result', message = '消极')
        else:
            tc=tkinter.messagebox.showinfo(title='result', message = '中立')
    tc
button1=Button(myWindow,compound=CENTER,image=predict_logo,command=get_result,width=100,height=100)
button1.place(x=100,y=250)
#进入消息循环
myWindow.mainloop()

D:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
